# Fit plane 
Perform least squares regression for a given point cloud

### _Issue_:
Currently, the plane fits are moved in space. 

This notebook will first show this for all the planes, then investigate what can be done to fix this issue.

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.extend([
    'C:/Users/Haakon/OneDrive/Dokumenter/FORSKNING/mastersproject/src/mastersproject',
    'C:/Users/Haakon/OneDrive/Dokumenter/FORSKNING/mastersproject/src/mastersproject/GTS'
])

In [2]:
import numpy as np
import pandas as pd

import GTS as gts

In [3]:
cls = gts.ISCData()

In [4]:
df = cls._full_structure_geometry()

In [5]:
df.head()

,depth,azimuth_struc,dip,aperture,type,borehole,x,y,z,length,...,shearzone,_trig_x,_trig_y,_trig_z,x_swiss,y_swiss,z_swiss,x_gts,y_gts,z_gts
0,1.76,239.29,86.44,1.89,Fracture,SBH3,667468.567,158885.383,1733.960,20.55,...,NaN,-0.174210,-0.980982,0.085591,667468.260391,158883.656472,1734.110640,68.260391,83.656472,34.110640
1,2.11,78.11,21.13,0.00,Fracture,PRP3,667468.390,158892.660,1733.100,32.33,...,NaN,-0.382879,0.821461,-0.422618,667467.582126,158894.393282,1732.208275,67.582126,94.393282,32.208275
2,2.26,252.34,77.18,1.83,Fracture,SBH3,667468.567,158885.383,1733.960,20.55,...,NaN,-0.174210,-0.980982,0.085591,667468.173286,158883.165981,1734.153435,68.173286,83.165981,34.153435
3,2.35,169.30,57.47,334.61,Minor ductile Shear-zone,FBS1,667466.424,158888.882,1732.782,44.80,...,NaN,-0.578330,0.621269,-0.528735,667465.064926,158890.341981,1731.539474,65.064926,90.341981,31.539474
4,2.54,134.77,59.79,10.06,Quartz,GEO3,667470.923,158912.008,1732.416,30.10,...,NaN,-0.671720,-0.000469,-0.740805,667469.216830,158912.006809,1730.534356,69.216830,112.006809,30.534356


In [6]:
_mask = df.shearzone.notna()
data = df.loc[_mask, ['type', 'borehole', 
                      'shearzone', 'x_gts', 
                      'y_gts', 'z_gts']]
data.head()

,type,borehole,shearzone,x_gts,y_gts,z_gts
151,S3 Shear-zone,PRP3,S3_1,62.428576,105.450142,26.519834
199,S1 Shear-zone,GEO3,S1_1,58.408848,111.999263,18.614810
216,S3 Shear-zone,PRP3,S3_2,60.985123,108.547049,24.926563
218,S1 Shear-zone,FBS3,S1_1,61.407644,114.369947,20.486618
220,S1 Shear-zone,GEO4,S1_1,54.635220,112.008451,21.640498


In [7]:
raw_coords = {}
for sz in data.shearzone.unique():
    raw_coords[sz] = data.loc[data.shearzone==sz, ('x_gts', 'y_gts', 'z_gts')].to_numpy()

## Use the `convex_plane` method to load shearzone vertices

In [8]:
shearzones = gts.convex_plane(shearzone=None, coords='gts')
shearzones.keys()

dict_keys(['S1_1', 'S1_2', 'S1_3', 'S3_1', 'S3_2'])

In [9]:
# Get vertices only
sz_vertices = {}
sz_proj = {}
for sz in shearzones:
    sz_vertices[sz] = shearzones[sz]['vertices'].T
    sz_proj[sz] = shearzones[sz]['proj'].T

In [10]:
# projected coordinates
sz_proj


{'S1_1': array([[ 87.80606576,  61.33271767,  34.06403553],
        [ 90.82921356,  63.66143038,  35.94864115],
        [ 84.55033821,  60.44929596,  37.36189694],
        [ 88.17271714,  62.69536022,  37.83521887],
        [ 81.11402366,  57.15920124,  33.11057374],
        [ 86.33461738,  58.39956091,  27.24450689],
        [ 78.92552538,  55.66141613,  32.36284444],
        [ 81.6052199 ,  55.16937582,  25.6501748 ],
        [ 75.2066799 ,  52.79661162,  30.04389059],
        [102.24710506,  74.26685673,  49.00341124],
        [ 38.92030081,  38.05869544,  50.75685593]]),
 'S1_2': array([[ 81.50944777,  65.18925885,  34.30507804],
        [ 76.54515764,  64.2444141 ,  38.79833719],
        [ 85.20676613,  70.84140274,  41.7250113 ],
        [ 83.84294681,  65.02226226,  30.86334619],
        [ 77.7347752 ,  64.14819374,  37.01962365],
        [ 75.27519296,  62.29597632,  36.23443562],
        [ 71.47127615,  59.07461728,  34.24384432],
        [ 78.67757417,  60.13793198,  27.05064

In [11]:
# Original coordinates
raw_coords

{'S3_1': array([[ 62.42857622, 105.4501422 ,  26.51983366],
        [ 59.95800966, 103.21328521,  18.87580234],
        [ 55.68524143, 108.31894614,  35.68536248],
        [ 54.93096116, 104.12516337,  21.37497152],
        [ 52.92578893, 103.38240836,  20.44133328],
        [ 55.54690593, 102.37530675,  16.16965004],
        [ 48.10827729, 103.37544387,  17.24416948],
        [ 49.86033668, 100.56931406,   6.76367706],
        [ 72.094     , 106.617     ,  32.813     ],
        [ 22.42      , 113.208     ,  33.608     ]]),
 'S1_1': array([[ 58.40884847, 111.99926347,  18.61481037],
        [ 61.40764443, 114.36994694,  20.48661828],
        [ 54.63522029, 112.00845129,  21.6404976 ],
        [ 58.59978767, 113.66474862,  22.29365105],
        [ 50.78924812, 109.42440764,  17.17388556],
        [ 56.35671854, 110.06692009,  11.49011407],
        [ 48.44373512, 108.19723976,  16.34363975],
        [ 51.23287523, 107.51656832,   9.68848744],
        [ 43.78967103, 106.94429858,  13.53319

In [12]:
residuals = {}
for sz in raw_coords:
    residuals[sz] = sz_proj[sz] - raw_coords[sz]

In [13]:
residuals

{'S3_1': array([[-11.84782914, -91.60503221,  31.94383512],
        [-11.86023412, -91.70094509,  31.97728115],
        [-11.71154421, -90.55130453,  31.57638693],
        [-11.79095   , -91.16525414,  31.79047893],
        [-11.71408379, -90.57094011,  31.5832341 ],
        [-11.80742124, -91.29260646,  31.8348883 ],
        [-11.76917201, -90.9968711 ,  31.73176163],
        [-11.89133031, -91.94137453,  32.06112195],
        [-11.87310241, -91.80044007,  32.01197631],
        [-11.86050619, -91.70304865,  31.97801469]]),
 'S1_1': array([[ 29.39721729, -50.6665458 ,  15.44922516],
        [ 29.42156913, -50.70851656,  15.46202287],
        [ 29.91511792, -51.55915533,  15.72139934],
        [ 29.57292948, -50.96938841,  15.54156782],
        [ 30.32477554, -52.2652064 ,  15.93668818],
        [ 29.97789884, -51.66735918,  15.75439282],
        [ 30.48179026, -52.53582363,  16.01920469],
        [ 30.37234467, -52.3471925 ,  15.96168736],
        [ 31.41700887, -54.14768696,  16.51069

In [14]:
data.groupby('shearzone', as_index=False).mean().sort_values('shearzone')

,shearzone,x_gts,y_gts,z_gts
0,S1_1,51.456704,109.806622,20.010900
1,S1_2,47.969502,113.118832,16.959744
2,S1_3,44.821713,117.088891,14.749656
3,S3_1,53.395810,105.063501,22.949580
4,S3_2,52.491075,108.564742,21.878384


## Note to the above results
As we can see, the residuals for each plane is large and equal in each coordinate. However, the shift does not correspond to the mean of the original points. Thus, we will have to explore other ways to compute the least squares plane

# Method 1: Fit plan to points in 3D perpendicular to the main axis
Source: http://www.ilikebigbits.com/2015_03_04_plane_from_points.html

Consider the equation of a plane $ax+by+cz+d = 0$. \ 
Simplify to the equation $ax+by+d = -z$, and solve for $a,b,d$.

This is the same as
\begin{equation}
\begin{bmatrix}
x_0 & y_0 & 1 \\
x_1 & y_1 & 1 \\
 & \dots & \\
x_n & y_n & 1
\end{bmatrix}
\begin{bmatrix}a\\ b\\ d\end{bmatrix}
= \begin{bmatrix}
-z_0 & -z_1 & \dots & -z_n
\end{bmatrix}
\end{equation}

To perform linear least squares, we multiply both sides with the transpose of the matrix, and multiply out to get

\begin{equation}
\begin{bmatrix}
\sum x_i x_i & \sum x_i y_i & \sum x_i \\
\sum y_i x_i & \sum y_i y_i & \sum y_i \\
\sum x_i & \sum y_i & N \\
\end{bmatrix}
\begin{bmatrix} a \\ b \\ d \\ \end{bmatrix}
= - \begin{bmatrix}
\sum x_i z_i \\
\sum y_i z_i \\
\sum z_i
\end{bmatrix}
\end{equation}
where $N$ is the number of points.

If we now define $x,y,z$, above relative to the centroid of the point cloud, we get $\sum x_i = \sum y_i = \sum z_i = 0$ and by inspecting the bottom row, we see $N\cdot d = 0 \implies d=0$.

This reduces the system to
\begin{equation}
\begin{bmatrix}
\sum x_i x_i & \sum x_i y_i \\
\sum y_i x_i & \sum y_i y_i \\
\end{bmatrix}
\begin{bmatrix} a \\ b \\ \end{bmatrix}
= - \begin{bmatrix}
\sum x_i z_i \\
\sum y_i z_i \\
\end{bmatrix}
\end{equation}

Using Cramer's rule, we get:
\begin{align}
D = \sum xx \times \sum yy - \sum xy \times \sum xy \\
a = \sum yz \times \sum xy - \sum xz \times \sum yy \\
b = \sum xy \times \sum xz - \sum xx \times \sum yz \\
n = [a,b,D]^T
\end{align}

Finally, originally, we assumed that the z-component of the plane normal is non-zero. If this assumption fails, the deteminant $D$ becomes zero. To mitigate this potential, issue, we do three separate calculations varying assuming each of $x,y,z$ be non-zero, and pick the one with the largest determinant.

Note: This method minimizes the squares of the residuals perpendicular to the main axis, not the residuals perpendicular to the plane.

In [15]:
data.head()

,type,borehole,shearzone,x_gts,y_gts,z_gts
151,S3 Shear-zone,PRP3,S3_1,62.428576,105.450142,26.519834
199,S1 Shear-zone,GEO3,S1_1,58.408848,111.999263,18.614810
216,S3 Shear-zone,PRP3,S3_2,60.985123,108.547049,24.926563
218,S1 Shear-zone,FBS3,S1_1,61.407644,114.369947,20.486618
220,S1 Shear-zone,GEO4,S1_1,54.635220,112.008451,21.640498


In [16]:
def fit_normal_to_points(points):
    """ Compute a normal from a collection of points.
    
    Source: http://www.ilikebigbits.com/2015_03_04_plane_from_points.html
    
    Parameters:
    points : np.ndarray (3,n)
        Array of points
    
    Returns
    normal : np.array (3,)
        Normalized normal vector to plane
    """
    assert((points.shape[0] == 3 ) & (points.shape[1] >= 3))
    pts = points.copy()
    N = pts.shape[1]
    
    # Calculate centroid and shift points
    centroid = np.atleast_2d(np.sum(pts, axis=1)/N).T
    pts = pts - centroid
    x, y, z = pts[0], pts[1], pts[2]
    
    # Compute the dot products, x*x, x*y, x*z, y*y, y*z, z*z
    xx = np.dot(x, x)
    xy = np.dot(x, y)
    xz = np.dot(x, z)
    yy = np.dot(y, y)
    yz = np.dot(y, z)
    zz = np.dot(z, z)
    
    # Compute the determinants
    det_x = yy*zz - yz*yz
    det_y = xx*zz - xz*xz
    det_z = xx*yy - xy*xy
    
    det_max = max(det_x, det_y, det_z)
    if det_max <= 0.0:
        return None  # The points do not span a plane
    
    # Pick path with best conditioning
    if det_x == det_max:
        normal = np.array([det_x, xz*yz - xy*zz, xy*yz - xz*yy])
    elif det_y == det_max:
        normal = np.array([xz*yz - xy*zz, det_y, xy*xz - yz*xx])
    else:  # det_z == det_max
        normal = np.array([xy*yz - xz*yy, xy*xz - yz*xx, det_z])
    
    return normal / np.sqrt((normal**2).sum())  # Normalize

In [17]:
normals = {}  # use raw_coords dictionary
for sz in raw_coords:
    normals[sz] = fit_normal_to_points(raw_coords[sz].T)
normals

{'S3_1': array([ 0.12191217,  0.94043104, -0.31737499]),
 'S1_1': array([-0.48531374,  0.86154042, -0.14905932]),
 'S3_2': array([ 0.14661264,  0.94276789, -0.29948862]),
 'S1_2': array([-0.49614612,  0.84118625, -0.21504585]),
 'S1_3': array([-0.50272651,  0.81396624, -0.29107562])}

In [18]:
old_normals = {}
for sz in shearzones:
    old_normals[sz] = shearzones[sz]['n']
old_normals

{'S1_1': array([-0.48526028,  0.83635338, -0.25502057]),
 'S1_2': array([-0.4825439 ,  0.79583676, -0.36578031]),
 'S1_3': array([-0.4894887 ,  0.79355907, -0.36147588]),
 'S3_1': array([ 0.12122318,  0.93727326, -0.32683906]),
 'S3_2': array([ 0.14065614,  0.92319817, -0.35766045])}

### Comparing the values of normals of the old and new method
We see that they are relatively similar (sometimes $\pm 0.05$) in one coordinate direction. \
This may be due to the choice of conditioning.

### Method 1 continued: 
Implement a method to project the original points to the fitted plane

In [19]:
def plane_from_points(points: np.ndarray) -> np.ndarray : 
    """ Compute a plane for a given point cloud.
    
    Compute the fitted plane to a point cloud, returning
    the points projected to the plane
    
    Parameters:
    points : nd.ndarray (3, n)
        Array of n 3D points
        
    Returns:
    proj : nd.ndarray (3, n)
        Array of n 3D points, projected to the best fit plane
        
    """
    assert((points.shape[0] == 3) & (points.shape[1] >= 3)), "Wrong input shape."
    
    N = points.shape[1]
    centroid = np.sum(points, axis=1)/N
    pointsR = (points.T - centroid).T
    normal = fit_normal_to_points(pointsR)
    assert(np.isclose(np.sqrt((normal**2).sum()), 1))  # Assert normalized

    proj = centroid + pointsR.T - np.outer(np.dot(pointsR.T, normal), normal)
    
    # Relative error of projected points
    error = np.sqrt(((proj - points.T)**2).sum(axis=1))
    length = np.sqrt(((points.T)**2).sum(axis=1))
    rel_error = error / length
    print(f"Sum of pointwise relative errors: {rel_error.sum():.4f}")
    
    return proj.T
    

Evaluate the method on `S1_1`:

In [20]:
points = raw_coords['S1_1'].T
proj = plane_from_points(points)
proj

Sum of pointwise relative errors: 0.0724


array([[ 57.78918931,  60.93749701,  54.68932337,  58.36539573,
         50.99187429,  55.92785431,  48.7457588 ,  51.07481105,
         44.86729617,  73.15489593,   9.47985294],
       [113.09929706, 115.20456376, 111.91240624, 114.08084673,
        109.06470087, 110.82825002, 107.66108018, 107.7971676 ,
        105.03127297, 124.38031623,  88.81294309],
       [ 18.42448819,  20.34221715,  21.65711482,  22.22165988,
         17.23612019,  11.35839266,  16.43640333,   9.63993959,
         13.864179  ,  33.59875229,  35.3406341 ]])

In [22]:
df = gts.convex_plane2(None)
df

Sum of pointwise relative errors: 0.0724
Sum of pointwise relative errors: 0.1129
Sum of pointwise relative errors: 0.0917
Sum of pointwise relative errors: 0.0375
Sum of pointwise relative errors: 0.0840


,x_proj,y_proj,z_proj,shearzone
0,9.479853,88.812943,35.340634,S1_1
1,73.154896,124.380316,33.598752,S1_1
2,55.927854,110.828250,11.358393,S1_1
3,51.074811,107.797168,9.639940,S1_1
4,10.869387,95.697726,34.410363,S1_2
5,75.477211,133.764400,34.253382,S1_2
6,53.895136,114.475229,8.594097,S1_2
7,48.422586,109.952285,3.527950,S1_2
8,18.393660,107.943321,34.819690,S1_3
9,75.225210,142.691685,33.834614,S1_3


# Method 2: plane fit with orthogonal distance regression
This variant uses the scipy package `orthogonal distance regression` \
See: https://docs.scipy.org/doc/scipy/reference/odr.html

* Proof of why centroid is contained in the plane of best fit: http://mathforum.org/library/drmath/view/63765.html
* Method in python using SVD: https://stackoverflow.com/questions/53591350/plane-fit-of-3d-points-with-singular-value-decomposition
* Another python implementation: https://stackoverflow.com/a/51132260/12580152
* SVD vs Basin-Hopping for plane fitting: https://stackoverflow.com/questions/47208838/best-fit-plane-to-3d-data-different-results-for-two-different-methods
* Line fitting discussion: https://zalo.github.io/blog/line-fitting/
* Explanation of decomposition matrices for SVD: https://math.stackexchange.com/questions/2810048/plane-fitting-using-svd-normal-vector

* Fitting plane to noisy points in 3D
    - https://www.ilikebigbits.com/2015_03_04_plane_from_points.html
    - https://www.ilikebigbits.com/2017_09_25_plane_from_points_2.html